In [1]:
import tensorflow as tf
import numpy as np
import scipy.io.wavfile
import os
tf.reset_default_graph()
sess = tf.InteractiveSession()

In [2]:
# data preprocessing
window = 10
n_train = 32000; n_test = 32000;
num_channels = 2
input_size = num_channels*window #n_channels*merge_factor
num_classes = 2 #n_directions
x = tf.placeholder(tf.float32, [None, input_size])
labels = tf.placeholder(tf.int32, [None])
offset = 63
train_x = []; train_t = []
test_x = []; test_t = []
for i in range(num_classes):
    temp_data = scipy.io.wavfile.read('noise3/'+'d'+str(0+(180*i))+'.wav')[1][:,::3]
    for j in range(n_train+n_test):
        if j%(2) == 0:
            train_x.append(temp_data[offset+(j*window):offset+((j+1)*window)])
            train_t.append(i)
    #for j in range(n_train,n_train+n_test):
        else:
            test_x.append(temp_data[offset+(j*window):offset+((j+1)*window)])
            test_t.append(i)

train_x = np.array(train_x); train_t = np.array(train_t)
train_x = train_x.reshape(num_classes*n_train,input_size)
mu = np.mean(train_x)
sig = np.std(train_x)
train_x = (train_x - mu+0.0)/sig
test_x = np.array(test_x); test_t = np.array(test_t)
test_x = test_x.reshape(num_classes*n_test,input_size)
test_x = (test_x - mu+0.0)/sig

ValueError: total size of new array must be unchanged

In [21]:
print train_x.shape
print num_classes*n_train
print input_size

(64000, 20)
64000
20


In [22]:
#hyperparameters
hidden_shape = [5,10,10,10]
net_shape = [input_size] + hidden_shape + [num_classes]
num_lyr = len(hidden_shape)
act_fns = ['tanh','tanh','tanh']
learning_rate = tf.placeholder(tf.float32,[])

In [23]:
#parameters
params = {'W':{},'b':{}}
for lay_num in range(num_lyr):
    params['W'][lay_num] = tf.Variable(tf.random_normal([net_shape[lay_num], net_shape[lay_num+1]],stddev = 0.01))
    params['b'][lay_num] = tf.Variable(tf.random_normal([net_shape[lay_num+1]],stddev = 0.01))

In [24]:
#network graph
def act_fn(a,lay_num):
    if act_fns[lay_num] == 'tanh':
        return tf.tanh(a)
    elif act_fns[lay_num] == 'ReLu':
        return tf.nn.relu(x)
    elif act[lay_num] == 'ELU':
        return tf.nn.elu(x)
    
def MLP(x,params):
    z = x
    for lay_num in range(num_lyr-1):
        z = act_fn(tf.add(tf.matmul(z,params['W'][lay_num]),params['b'][lay_num]),lay_num)
    logits = tf.add(tf.matmul(z,params['W'][lay_num+1]),params['b'][lay_num+1])
    return logits

In [25]:
y = MLP(x,params)
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(y,labels))
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
sess.run(tf.initialize_all_variables())

In [26]:
n_iter = 0
while True:
    _,cost_ = sess.run([optimizer,cost],feed_dict={x:train_x,labels:train_t,learning_rate:0.01})
    if n_iter % 100 == 0:
        print('iter'+str(n_iter)+' cost:'+str(cost_))
        y_train = sess.run(y,feed_dict={x:train_x})
        print 'train pf = ' + str((sum(np.argmax(y_train,axis=1) == train_t) + 0.0)/(n_train*num_classes))
        y_test = sess.run(y,feed_dict={x:test_x})
        print 'test pf = ' + str((sum(np.argmax(y_test,axis=1) == test_t) + 0.0)/(n_test*num_classes))+'\n'
#         mypath = 'vars/iter'+str(n_iter)
#         if not os.path.isdir(mypath):
#            os.makedirs(mypath)
#         myfile = mypath+'/'+str(n_iter)
#         saver.save(sess,myfile,write_meta_graph=False)
    n_iter += 1

iter0 cost:2.30155
train pf = 0.5
test pf = 0.5

iter100 cost:1.94122
train pf = 0.5
test pf = 0.5

iter200 cost:1.66042
train pf = 0.5
test pf = 0.5

iter300 cost:1.44876
train pf = 0.5
test pf = 0.5

iter400 cost:1.29078
train pf = 0.5
test pf = 0.5

iter500 cost:1.17178
train pf = 0.5
test pf = 0.5

iter600 cost:1.08047
train pf = 0.5
test pf = 0.5

iter700 cost:1.0091
train pf = 0.5
test pf = 0.5

iter800 cost:0.952633
train pf = 0.5
test pf = 0.5

iter900 cost:0.907697
train pf = 0.5
test pf = 0.5

iter1000 cost:0.871879
train pf = 0.5
test pf = 0.5

iter1100 cost:0.843305
train pf = 0.5
test pf = 0.5

iter1200 cost:0.82046
train pf = 0.5
test pf = 0.5

iter1300 cost:0.802123
train pf = 0.5
test pf = 0.5

iter1400 cost:0.787315
train pf = 0.5
test pf = 0.5

iter1500 cost:0.775269
train pf = 0.5
test pf = 0.5

iter1600 cost:0.765388
train pf = 0.5
test pf = 0.5

iter1700 cost:0.757213
train pf = 0.5
test pf = 0.5

iter1800 cost:0.75039
train pf = 0.5
test pf = 0.5

iter1900 cost:0.

Exception AssertionError: AssertionError("Nesting violated for default stack of <type 'weakref'> objects",) in <bound method InteractiveSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x7fa5b6d3f8d0>> ignored


KeyboardInterrupt: 

In [ ]:
sess.close()